# **UFO Sightings - Part 1: Preprocessing Data**

## 1. Importing Data Set and Necessary Packages

In [1]:
import html                                  # package used to convert text
import numpy as np                           # linear algebra
import os                                    # package used to set work directory
import pandas as pd                          # package used to create dataframes
import re                                    # regular expressions

from datetime import datetime                # package used to convert string to datetime 

In [2]:
data = pd.read_csv('scrubbed.csv', low_memory=False)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  object 
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  object 
 10  longitude             80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


The data set contains a decent amount of observations, 80,332 to be specific, and 11 columns. Apart from the column `longitude`, the columns are not assigned a data type. Before we assign data types, we will first do some further exploration. The columns `state`, `country`, `shape` and `comments` seem to have missing values which should be addressed as well. There are two different columns that contain the durations of the UFO sightings. All of these points will be addressed in the next section where we will be cleaning and preparing the data for analysis.

## 2. Data Cleaning

In the previous section, we've already noticed some things with the data. We will go through them one by one in this section. Whatever other issues we stumble on during the process will also be addressed. We'll copy the data set to a new variable so we can manipulate the data without losing the original raw data. We also print the first five rows to get a look of what the entries look like.

In [4]:
df = data.copy()
pd.set_option('display.max_columns', None)    # shows all columns
print(df.head())

           datetime                  city state country     shape  \
0  10/10/1949 20:30            san marcos    tx      us  cylinder   
1  10/10/1949 21:00          lackland afb    tx     NaN     light   
2  10/10/1955 17:00  chester (uk/england)   NaN      gb    circle   
3  10/10/1956 21:00                  edna    tx      us    circle   
4  10/10/1960 20:00               kaneohe    hi      us     light   

  duration (seconds) duration (hours/min)  \
0               2700           45 minutes   
1               7200              1-2 hrs   
2                 20           20 seconds   
3                 20             1/2 hour   
4                900           15 minutes   

                                            comments date posted    latitude  \
0  This event took place in early fall around 194...   4/27/2004  29.8830556   
1  1949 Lackland AFB&#44 TX.  Lights racing acros...  12/16/2005    29.38421   
2  Green/Orange circular disc over Chester&#44 En...   1/21/2008        53

### 2.1 Coordinates

There are two columns that contain the coordinates of the UFO sightings. The `longitude` column's data type is `float64` which seems appropriate for the type of variable. We would ideally also want the `latitude` to be a float type date. First, the column name for the longitude should be renamed as it for some reason has an extra space at the end reading `longitude ` instead of `longitude`.

In [5]:
df = df.rename(columns={'longitude ': 'longitude'})

When we try to convert the column `latitude`, it immediately returns a value error. One of the entries is not numerical. The recorded value is '33q.200088'. We look for this element in the column and print the corresponding column values to see if we can fix this issue.

In [6]:
latitude_error_index = np.where(df == '33q.200088')[0].tolist()[0]
print(df.iloc[latitude_error_index])

datetime                                                  5/22/1974 05:30
city                                         mescalero indian reservation
state                                                                  nm
country                                                               NaN
shape                                                           rectangle
duration (seconds)                                                    180
duration (hours/min)                                            two hours
comments                Huge rectangular object emmitting intense whit...
date posted                                                     4/18/2012
latitude                                                       33q.200088
longitude                                                     -105.624152
Name: 43782, dtype: object


We have the name of the location, namely, *Mescalero Indian Reservation*. A quick Google search gives us the following coordinates of this reservation (that nowadays is called *Mescalero Apache Reservation*): (33.157010803266594, -105.7795644741356). We can derive from this that the letter 'q' is accidentally added and we can just remove it.

In [7]:
df.loc[latitude_error_index, 'latitude'] = '33.200088'

Let's try again to convert the column to float.

In [8]:
df = df.astype({'latitude':'float'})
print(df.latitude.dtype)

float64


The conversion is successful. Now both coordinates are float variables. The final thing we will check if there are missing values. From previous section (3.0) we already know that there are no empty cells. Also, the columns are completely numerical. However, we should check for zero values which may indicate missing values because a perfect zero value is highly unlikely as we are dealing with coordinates.

In [9]:
print('Number of zero values in latitude:', sum(df['latitude']==0))
print('Number of zero values in longitude:', sum(df['longitude']==0))

Number of zero values in latitude: 0
Number of zero values in longitude: 0


No issues here, which is great.

### 2.2 Date/time data

There are two variables that contain date/time data: `datetime`, which is the time at which the sighting took place, and `date posted`, which is the time at which the comment was recorded. We will start with `date posted` since this one is a little less complicated as it only contains the date, not the time. We first change the column name by replacing the space with an underscore for convenience. Then we convert the data type to `datetime64`.

In [10]:
df = df.rename(columns={'date posted': 'date_posted'})
df['date_posted'] = [datetime.strptime(line, '%m/%d/%Y') for line in df['date_posted']]
print(df.date_posted.dtype)

datetime64[ns]


Now that we have converted `date_posted`, we move on to `datetime`. However, there is one issue with this column. The time is recorded in a 24-hour clock format. However, 12 a.m. (24h) is noted as '24:xx' while we need it to be '00:xx' to be able to convert the data type. Once that's fixed, we can do the same as above.

In [11]:
corrected_datetime = [line[:line.find('24:')] + '00:' + line[line.find('24:') + 3:] if line.find('24:') != -1 else line for line in df['datetime']]
df['datetime'] = [datetime.strptime(line, '%m/%d/%Y %H:%M') for line in corrected_datetime]
print(df.datetime.dtype)

datetime64[ns]


Now both columns are succesfully converted into datetime variables.

### 2.3 Duration

As mentioned before, there are two columns containing the durations of the UFO sightings: `duration (seconds)` and `duration (hours/min)`.

In [12]:
print(df['duration (seconds)'].head())
print(df['duration (hours/min)'].head())

0    2700
1    7200
2      20
3      20
4     900
Name: duration (seconds), dtype: object
0    45 minutes
1       1-2 hrs
2    20 seconds
3      1/2 hour
4    15 minutes
Name: duration (hours/min), dtype: object


`duration (hours/min)` is the textual recording of the duration. This seems to be the original raw data that someone (manually) converted into numerical notation in seconds. For time's sake, we will assume that this process is done properly. We ideally would want to solely use the `duration (seconds)` column and convert it to a float data type. However, when we try to convert the column, it returns a value error. A few entries have been somehow corrupted. What happened is that for a few entries the numerical part of the element is followed by a `. There are three elements this happend to: 
- 2`
- 8`
- 0.5`

We look those elements up and fix them so they are completely numerical.

In [13]:
wrong_elements = ["2`", "8`", "0.5`"]
for we in wrong_elements:
    duration_error_index = np.where(df == we)[0].tolist()[0]
    df.loc[duration_error_index, 'duration (seconds)'] = we[:-1]

Now we can convert the column into a float data type.

In [14]:
df = df.astype({'duration (seconds)':'float'})

Since we are not going to use the textual duration column and we can always access it from the raw data, we are going to remove this column and rename the numerical column.

In [15]:
df = df.drop(columns='duration (hours/min)')
df = df.rename(columns={'duration (seconds)': 'duration'})
df = df.astype({'duration':'float'})
print(df.duration.dtype)

float64


### 2.4 Shape

This column contains the shape of the UFO that was claimed to be sighted. Earlier we noticed that there were missing values: 1,931 in total. Let's take a look at how this column looks like. We'll begin with listing the unique entries.

In [16]:
df['shape'].value_counts()

shape
light        16565
triangle      7865
circle        7608
fireball      6208
other         5649
unknown       5584
sphere        5387
disk          5213
oval          3733
formation     2457
cigar         2057
changing      1962
flash         1328
rectangle     1297
cylinder      1283
diamond       1178
chevron        952
egg            759
teardrop       750
cone           316
cross          233
delta            7
round            2
crescent         2
pyramid          1
flare            1
hexagon          1
dome             1
changed          1
Name: count, dtype: int64

There are 30 different unique entries including the missing values. There is one shape called 'unknown'. Since there are way too many missing values to do some text analysis on the comments without the guarantee that enough of them contain information on the shape of the alleged UFO, we will just assign the value *'unknown'* to the missing values. The subset of observations with an *'unknown'* shape is already much larger than the amount of missing values, so this should not mess up the proportions of the different subsets too much.

In [17]:
df['shape'] = [line if not pd.isna(line) else 'unknown' for line in df['shape']]

### 2.5 Comments

The column containing the comments has 15 missing values. We will replace these with empty strings. We will simultaneously deal with the html encoding that has been applied to the text (example *'&'* was saved in the csv file as *'\&amp;'* and *"'"* as *'\&#x27;'*).

In [18]:
df['comments'] = [html.unescape(line) if not pd.isna(line) else '' for line in df['comments']]

This is all we will do to the comments. We will create a new column with the length of the comments which may be interested in the data analysis part of this project. The length of the comments will be the number of characters in the comments.

In [19]:
df['comments_length'] = [len(line) for line in df['comments']]

### 2.6 Location String Data

Finally, we will explore the location data: `city`, `state` and `country`. The latter two have missing values. Let's look at the number of unique values each of these columns contain.

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   datetime         80332 non-null  datetime64[ns]
 1   city             80332 non-null  object        
 2   state            74535 non-null  object        
 3   country          70662 non-null  object        
 4   shape            80332 non-null  object        
 5   duration         80332 non-null  float64       
 6   comments         80332 non-null  object        
 7   date_posted      80332 non-null  datetime64[ns]
 8   latitude         80332 non-null  float64       
 9   longitude        80332 non-null  float64       
 10  comments_length  80332 non-null  int64         
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 6.7+ MB


In [21]:
print('Number of unique values for city:', len(pd.unique(df['city'])))
print('Number of unique values for state:', len(pd.unique(df['state'])))
print('Number of unique values for country:', len(pd.unique(df['country'])))

Number of unique values for city: 19900
Number of unique values for state: 68
Number of unique values for country: 6


It's interesting to see that there are only six unique values for `country`. Let's see which countries these are.

In [22]:
print(df['country'].value_counts())

country
us    65114
ca     3000
gb     1905
au      538
de      105
Name: count, dtype: int64


There are 9,669 missing values. About 65,000 sightings took place in the United States. The other 5,548 sightings have been reported to have happened in Canada, Great Britain, Australia and Germany. Since we have the coordinates of the sightings, we will not spend too much time on these three columns. We will fix the missing values issue and assign a string data type to the columns.

In [23]:
df['state'] = [line if not pd.isna(line) else '' for line in df['state']]
df['country'] = [line if not pd.isna(line) else '' for line in df['country']]

## 2.7 Duplicates

Finally, we will check for duplicates and remove them if necessary.

In [24]:
sum(df.duplicated())

2

There are two rows that have duplicates. We will remove them.

In [25]:
df.drop_duplicates(inplace=True)

## 3. Summary

We have converted `datetime` and `date posted` (renamed to `date_posted`) into datetime data types. We have dealt with the missing values in the columns `state`, `country`, `shape` and `comments`. The text numerical have been converted into float or integer elements. The column `duration (hours/min)` has been dropped as `duration (seconds)` (renamed to `duration`) would be more appropriate to work with. We added a column `comments_length` containing the length of the comments. Below, you find an overview of the data set properties. Finally, two rows that were duplicates were removed.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80330 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   datetime         80330 non-null  datetime64[ns]
 1   city             80330 non-null  object        
 2   state            80330 non-null  object        
 3   country          80330 non-null  object        
 4   shape            80330 non-null  object        
 5   duration         80330 non-null  float64       
 6   comments         80330 non-null  object        
 7   date_posted      80330 non-null  datetime64[ns]
 8   latitude         80330 non-null  float64       
 9   longitude        80330 non-null  float64       
 10  comments_length  80330 non-null  int64         
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 7.4+ MB


We export the preprocessed data into a csv file so we can work on it in the next part.

In [27]:
df.to_csv('preprocessed_data.csv', index=False)